# Senate Scraper 2.0

This scraper is meant to be an improvement over the first one in a variety of ways. First, it's built using functions instead of spaghetti code. Second, it should be able to handle errors better than the original.

## TO DOs:

Turns out the dtg/signed/veto regexs don't...work. I need to walk away from this for a bit and take care of my other WWW assignments before giving this another wack.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import csv
import re
import pdb

In [2]:
## 2011 on s21, a20 (max: s7867)
## 2013 on s20, a21 (max: s7958)
## 2015 on s11, a21 (max: s8236)
## 2017 on s17, a11 (max: s9199)
## 2019 on s11, a11 (max: s9115)

legis_meta_data = [{"session":2011,"start_number":21,"end_number":7867},
{"session":2013,"start_number":20,"end_number":7958},
{"session":2015,"start_number":11,"end_number":8236},
{"session":2017,"start_number":17,"end_number":9199},
{"session":2019,"start_number":11,"end_number":9115}]

legis_meta_data_TEST = [{"session":2011,"start_number":21,"end_number":31},
{"session":2013,"start_number":20,"end_number":30},
{"session":2015,"start_number":11,"end_number":21},
{"session":2017,"start_number":17,"end_number":27},
{"session":2019,"start_number":11,"end_number":21}]

legis_meta_data_TEST_two = [{"session":2011,"start_number":21,"end_number":221},
{"session":2013,"start_number":20,"end_number":220},
{"session":2015,"start_number":11,"end_number":211},
{"session":2017,"start_number":17,"end_number":217},
{"session":2019,"start_number":11,"end_number":211}]

legis_meta_data_TEST_three = [{"session":2011,"start_number":121,"end_number":221},
{"session":2013,"start_number":120,"end_number":220},
{"session":2015,"start_number":111,"end_number":211},
{"session":2017,"start_number":117,"end_number":217},
{"session":2019,"start_number":111,"end_number":211}]

######## Session splits ########

legis_meta_data_session2011 = [{"session":2011,"start_number":21,"end_number":7867}]

legis_meta_data_session2013 = [{"session":2013,"start_number":20,"end_number":7958}]

legis_meta_data_session2015 = [{"session":2015,"start_number":11,"end_number":8236}]

legis_meta_data_session2017 = [{"session":2017,"start_number":17,"end_number":9199}]

legis_meta_data_session2019 = [{"session":2019,"start_number":11,"end_number":9115}]

######## Random tests splits ########

legis_nonetype_TEST = [{"session":2015,"start_number":5000,"end_number":5020}]

legis_small_TEST = [{"session":2011,"start_number":79,"end_number":82}]

legis_dtg_TEST = [{"session":2011,"start_number":21,"end_number":25}]

legis_dtg_TEST2 = [{"session":2011,"start_number":4490,"end_number":4504}]

legis_lla_TEST = [{"session":2019,"start_number":8445,"end_number":8455}]

In [3]:
failed_scrape = []
failed_parse = []

def scrape_url(session, bill_counter):
    try:
        url = f"https://www.nysenate.gov/legislation/bills/{session}/s{bill_counter}"
        #url = "https://www.nysenate.gov/legislation/bills/2011/s966"
        raw_html = requests.get(url).content
        soup = BeautifulSoup(raw_html, "html.parser")
        scrape_dict = {"soup":soup,"url":url}
        return scrape_dict
    except Exception as e:
        failed_scrape_transfer = []
        failed_scrape_transfer.append(url)
        failed_scrape_transfer.append(e)
        
        failed_scrape.append(failed_scrape_transfer)
        
        print(e)
    
def parse_data(soup, url, bill_counter):
    try:
        transfer_list = []
        
        try:
            ##Bill number

            bill_number = soup.find(class_="nys-title c-bill-title").text.strip()
            transfer_list.append(bill_number)
        except:
            transfer_list.append("bill no. append FAIL")
            
        try:
            ## Session year
            
            transfer_list.append(session)
        except:
            transfer_list.append("session append FAIL")
        #########################
        try:
            #Append URL
            transfer_list.append(url)
        except:
            transfer_list.append("url append FAIL")
        #########################
        try:
            #Append bill title
            bill_title = soup.find(class_="c-detail--header-meta").text.strip()
            transfer_list.append(bill_title)
        except:
            transfer_list.append("bill title append FAIL")
        #########################
        try:
            ## Sponser and district
            spon_sen_name = soup.find("h4").text.strip()
            transfer_list.append(spon_sen_name)
        except:
            transfer_list.append("No sponsers found")
        #########################
        try:
            dist_name = soup.find(class_="nys-senator--district").text.strip()
            transfer_list.append(dist_name)
        except:
            transfer_list.append("No district found")
        #########################
        ## Vote count

        #transfer_list.append("No aye votes for this bill.")
        
        vote_regex = re.compile('.*(\S\S\S\S)')
        floor_vote_regex = re.compile('.*(\S\S\S\S\S\s\S\S\S\S)')
        
        try:
            vote_soup = soup.find(class_="c-bill--vote c-bill--vote_1").find\
            (class_="c-detail--subhead c-detail--section-title c-bill-detail--subhead")
            if vote_regex.match(vote_soup.text.strip()):    
                try:
                    if floor_vote_regex.match(vote_soup.text.strip()):
                        aye_vote = soup.find_all(class_="vote-count")
                        transfer_list.append(aye_vote[0].text.strip())
                    else:
                        transfer_list.append("Not brought to floor")
                except Exception as e:
                    transfer_list.append(e)
            #########################
                try:
                    if floor_vote_regex.match(vote_soup.text.strip()):
                        nay_vote = soup.find_all(class_="vote-count")
                        transfer_list.append(nay_vote[1].text.strip())
                    else:
                        transfer_list.append("Not brought to floor")
                except Exception as e:
                    transfer_list.append(e)
#             else:
#                 transfer_list.append("Not brought to floor")
#                 transfer_list.append("Not brought to floor")
            #########################              

        except:
                transfer_list.append("Not voted on")
                transfer_list.append("Not voted on")                
        ## transferred, signed, vetoed

        #pdb.set_trace()

        try:
            target = soup.select_one\
            ('.cbill--actions-table--row:has(td:contains("delivered to governor"))')
            if target:
                #print(target.select_one('.c-bill--actions-table-col1').text)
                #dtg
                transfer_list.append('TRUE')
                
                #dtg_date
                transfer_list.append(target.select_one('.c-bill--actions-table-col1').text)
                
                #lla_needed?
                transfer_list.append('TRUE')
                
                #lla date
                transfer_list.append(target.find_next('tr').select_one\
                                     ('.c-bill--actions-table-col1').text)
                
                #lla_action
                transfer_list.append(target.find_next('tr').select_one\
                                     ('.c-bill--actions-table-col2 span').text)
                
                #dtg text
                #transfer_list.append(target.select_one
                #('.c-bill--action-line-assembly:last-of-type, .c-bill-
                #-action-line-senate:last-of-type').text)
                
                
            
            else:
                #dtg
                transfer_list.append("FALSE")
                #dtg_date
                transfer_list.append("FALSE")
                #lla_needed
                transfer_list.append("FALSE")
                #lla_date
                transfer_list.append("FALSE")
                #lla_action
                transfer_list.append("FALSE")
                
        except Exception as e:
            transfer_list.append(Exception)
            transfer_list.append(Exception)


        try:
            target_signed = soup.select_one('.cbill--actions-table--row td:contains("signed")')
            if target_signed:
                transfer_list.append('true')
                transfer_list.append(target_signed.parent.td.text)     

            else:
                transfer_list.append("false")
                transfer_list.append("no date")

        except Exception as e:
            transfer_list.append(e)

        try:
            target_veto = soup.select_one('.cbill--actions-table--row td:contains("vetoed")')
            if target_veto:
                transfer_list.append("true")
                transfer_list.append(target_veto.parent.td.text)

            else:
                transfer_list.append("false")
                transfer_list.append("no date")
        except Exception as e:
            transfer_list.append(e)
        try:
            sponser_memo = soup.find(class_="c-bill-fulltext").text.strip()
            transfer_list.append(sponser_memo)
        except Exception as e:
            transfer_list.append(e)
            
        try:
            sponser_memo2 = soup.find(class_="c-text--preformatted").find_all\
            (class_="c-bill-fulltext")
            transfer_list.append(sponser_memo2[1].text.strip())
        except Exception as e:
            transfer_list.append(e)
            
        try:
            bill_text = soup.find(class_="c-bill-text__bill").text.strip()
            transfer_list.append(bill_text)
        except Exception as e:
            transfer_list.append(e)
        return transfer_list
    
    except Exception as e:
        failed_parse_transfer = []
        failed_parse_transfer.append(bill_counter)
        failed_parse_transfer.append(url)
        failed_parse_transfer.append(e)
        
        failed_parse.append(failed_parse_transfer)

In [4]:
df = pd.DataFrame()
df_failed_scrape = pd.DataFrame()
df_failed_parse = pd.DataFrame()

for i in legis_meta_data_session2011:
    session = i['session']
    end = i['end_number']
    counter = i['start_number']
    
    while counter <= end:
        
        ## Initilize list
        bill_info = []
        
        ## Utlize functions
        html_markup_data = scrape_url(session, counter)
        parsed_bill_data = parse_data(html_markup_data['soup'], html_markup_data['url'], counter)
        
        ## Append successfully parsed bill data
        bill_info.append(parsed_bill_data)
        df = df.append(bill_info)
    
        counter += 1
        time.sleep(1)
        pass

## Append failed scrape data
df_failed_scrape = df_failed_scrape.append(failed_scrape)

## Append failed parse data
df_failed_parse = df_failed_parse.append(failed_parse)

In [5]:
df.columns = ["bill_number","session", "url","bill_title","spon_sen_name",\
              "dist_name","aye_vote","nay_vote",\
              "dtg","dtg_date","lla_needed?","lla_date","lla_action",\
              "signed_by_gov","signed_date","vetoed_by_gov","vetoed_date",\
              "sponser_memo","sponser_memo2","bill_text"]

In [6]:
df.to_csv("2011_legis_data_4.12.2021.csv",sep='|',index=False,header=["bill_number","session",\
                "url","bill_title","spon_sen_name",\
              "dist_name","aye_vote","nay_vote",\
              "dtg","dtg_date","lla_needed?","lla_date","lla_action",\
              "signed_by_gov","signed_date","vetoed_by_gov","vetoed_date",\
              "sponser_memo","sponser_memo2","bill_text"])

### Parsing errors and failed URL scrapes

In [7]:
try:
    df_failed_scrape.to_csv("failed_scrape2011.csv",index=False,header=["url","error_message"])
except:
    print("No urls failed to scrape!")

No urls failed to scrape!


In [8]:
try:
    df_failed_parse.to_csv("failed_parse2011.csv",index=False,header=["bill_counter","url","error_message"])
except:
    print("No urls failed to parse!")

No urls failed to parse!


In [9]:
df.head()

,bill_number,session,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,dtg,dtg_date,lla_needed?,lla_date,lla_action,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text
0,senate Bill S21,2011,https://www.nysenate.gov/legislation/bills/201...,Relates to payment of independent contractors,Daniel L. Squadron,"(D, WF) 0 Senate District",Not voted on,Not voted on,FALSE,FALSE,FALSE,FALSE,FALSE,false,no date,false,no date,BILL NUMBER:S21\n\nTITLE OF BILL:\nAn act\nto ...,-Provides that the Department of Labor must pr...,S21 (ACTIVE) - Bill Text\n \ndownload...
0,senate Bill S22,2011,https://www.nysenate.gov/legislation/bills/201...,Provides that the income level for the require...,Daniel L. Squadron,"(D, WF) 0 Senate District",59,0,FALSE,FALSE,FALSE,FALSE,FALSE,false,no date,false,no date,BILL NUMBER:S22\n\nTITLE OF BILL:\nAn act\nto ...,"income persons, particularly senior citizens. ...",S22 (ACTIVE) - Bill Text\n \ndownload...
0,senate Bill S23,2011,https://www.nysenate.gov/legislation/bills/201...,"Relates to the sale of tonic water, bitters an...",Daniel L. Squadron,"(D, WF) 0 Senate District",Not voted on,Not voted on,FALSE,FALSE,FALSE,FALSE,FALSE,false,no date,false,no date,BILL NUMBER:S23\n\nTITLE OF BILL:\nAn act\nto ...,Section 3- Effective date.\n\nJUSTIFICATION:\n...,S23 (ACTIVE) - Bill Text\n \ndownload...
0,senate Bill S24,2011,https://www.nysenate.gov/legislation/bills/201...,Requires certification notice to tenant when r...,Daniel L. Squadron,"(D, WF) 0 Senate District",Not voted on,Not voted on,FALSE,FALSE,FALSE,FALSE,FALSE,false,no date,false,no date,BILL NUMBER:S24\n\nTITLE OF BILL:\nAn act\nto ...,prevent such disputes from being elevated to t...,S24 (ACTIVE) - Bill Text\n \ndownload...
0,senate Bill S25,2011,https://www.nysenate.gov/legislation/bills/201...,Relates to making the provisions governing liq...,Daniel L. Squadron,"(D, WF) 0 Senate District",Not voted on,Not voted on,FALSE,FALSE,FALSE,FALSE,FALSE,false,no date,false,no date,BILL NUMBER:S25\n\nTITLE OF BILL:\nAn act\nto ...,EXISTING LAW:\nABC Law § 64{6-a) sets forth ce...,S25 (ACTIVE) - Bill Text\n \ndownload...


In [10]:
df_failed_parse.shape

(0, 0)

In [11]:
df_failed_scrape.head()

""


# Testing ground

In [12]:
#         if dtg in check_list:

#             i = check_list.index(dtg)
#             ## Only putting this note here because I changed it when I should not have. It's a plus and not a minus
#             ## because I'm looking for when the legislature finished voting on the merits of the bill, not when it
#             ## was actually transmitted (since once it's transmitted, the governor has a set number of days to do something
#             ## with it). I'll have to check this later, but I THINK once the legislature passes a bill, it gets it's
#             ## dtg designation - which makes it looks like the bill has been sitting on Cuomo's desk for months when
#             ## in reality the legislature hasn't sent the bill yet.
#             transfer_list.append(check_list[i+1]) ## last legislative action date (not counting dtg)
#             transfer_list.append(check_list[i-1]) ## dtg date
            
#         elif any(dtg_regex.match(dtg_check_list) for dtg_check_list in check_list):
#             transfer_list.append(check_list[4])
#             transfer_list.append(check_list[2])

In [13]:
# url = "https://www.nysenate.gov/legislation/bills/2011/s1415"
# raw_html = requests.get(url).content
# soup = BeautifulSoup(raw_html, "html.parser")
# print(raw_html)


In [14]:
# import os

In [15]:
# with open('soup.txt', 'w') as data_file:
#     data_file.write(str(soup.encode('utf-8')))
# os.startfile('soup.txt')

In [16]:
#        print(check_list)
        
        
#       
#         try:
#             target_delivered = soup.select_one('.cbill--actions-table--row td:contains("delivered")')
#             if target_delivered:
#                 #i = check_list.inde
#                 transfer_list.append(target_delivered.parent.td.text) #last legis action date (not counting dtg)
#                 transfer_list.append(target_delivered.parent.td.text) #dtg date
                
#         except Exception as e:
#             transfer_list.append(e)
#             transfer_list.append(e)
                
#         
#         dtg = "delivered to governor"
#         dtg_regex = re.compile('/.*(\S\S\S\S\S\S\S\S\S\s\S\S\s\S\S\S\S\S\S\S\S).*')
        
#         if dtg in check_list:

#             i = check_list.index(dtg)
#             ## Only putting this note here because I changed it when I should not have. It's a plus and not a minus
#             ## because I'm looking for when the legislature finished voting on the merits of the bill, not when it
#             ## was actually transmitted (since once it's transmitted, the governor has a set number of days to do something
#             ## with it). I'll have to check this later, but I THINK once the legislature passes a bill, it gets it's
#             ## dtg designation - which makes it looks like the bill has been sitting on Cuomo's desk for months when
#             ## in reality the legislature hasn't sent the bill yet.
#             transfer_list.append(check_list[i+1]) ## last legislative action date (not counting dtg)
#             transfer_list.append(check_list[i-1]) ## dtg date
            
#         elif any(dtg_regex.match(dtg_check_list) for dtg_check_list in check_list):
#             transfer_list.append(check_list[4])
#             transfer_list.append(check_list[2])
            
#         else:
#             transfer_list.append("no floor vote")
#             transfer_list.append("not delivered to governor")
#####
    
        
        
    

In [17]:
#             dtg_counter = 0
#             for row in soup.find_all('tr'):
            
#             # check to see if it was delivered to governor
#                 if 'delivered to governor' in row.get_text():
#                     # if it is, we convert the cells (td) into a list, so that we can grab it
#                     # the structure looks like it's always cell[0] = date, cell[1] = text, so we define them
#                     cells = row.find_all('td')
#                     cdate = cells[0].get_text()
#                     ctext = cells[1].get_text()
#                     # there's an issue here where there's sometimes a lot of text in cell[1]
#                     # we might need to do more parsing and cleanup
#                     transfer_list.append('true')
#                     transfer_list.append(cdate)
#                     dtg_counter = dtg_counter + 1
                    
#             if dtg_counter < 1:
#                 transfer_list.append("false")
#                 transfer_list.append("no date")
                
#                 transfer_list.append("false")
#                 transfer_list.append("no date")
                
#             if dtg_counter > 0:
#                 tablebody=soup.select_one(".table.c-bill--actions-table > tbody")
#                 check_list = [item.text.strip() for item in tablebody.select("td")]
#                 lla_date = check_list[4]
#                 lla = check_list[5]
#                 transfer_list.append("true")
#                 transfer_list.append(lla_date)
                
#                 i = check_list.index("delivered to governor")
#                 lla_date = check_list[-1]
#                 lla = check_list[-2]
#                 transfer_list.append("true")
#                 transfer_list.append(lla_date)

In [18]:
#         try:
#             target = soup.select_one('.cbill--actions-table--row:has(td:contains("delivered to governor"))')
#             if target:
#                 transfer_list.append(target.select_one('.c-bill--action-line-assembly:last-of-type, .c-bill--action-line-senate:last-of-type').text)
#                 transfer_list.append(target.select_one('.c-bill--actions-table-col1').text)
                
#             else:
#                 transfer_list.append("no floor vote")
#                 transfer_list.append("not delivered to governor")
                
#         except Exception as e:
#             transfer_list.append(e)
#             transfer_list.append(e)